In [ ]:
exclude_dirs = ("venv", "tmp", "user_data")
addon_name = "blenderai_zealo"
locales_to_make = ("zh_HANS", )

In [ ]:
import glob
import os
import subprocess


root_dir = os.path.normpath(r"C:\blenderProjects\blenderai_zealo")
py_files = glob.glob("**/*.py", recursive=True)
py_files = [
    file for file in py_files
    if not any([exclude_dir in file for exclude_dir in exclude_dirs])
]
pygettext_file = os.path.normpath(r"C:\Python\Python310\Tools\i18n\pygettext.py")
locale_dir = os.path.normpath(os.path.join(root_dir, "./locale"))
os.makedirs(locale_dir, exist_ok=True)
pot_filepath = os.path.normpath(os.path.join(locale_dir, f"./{addon_name}.pot"))

get_pot_file_ok = False
try:
    process = subprocess.run(
        ["xgettext", "-L", "Python","--keyword=_:1,2c", "-o", f"{pot_filepath}", *py_files], 
        check=True, 
        capture_output=True, 
        text=True
    )
    print(f"Get pot file success into {pot_filepath}")
    get_pot_file_ok = True
except Exception as e:
    print(f"Get pot file failed since: {e.stderr}")

if get_pot_file_ok:
    replace_charset_ok = False
    try:
        with open(pot_filepath, "r+t", encoding="utf-8") as f:
            content = f.read()
        new_content = content.replace("charset=CHARSET", "charset=UTF-8")
        with open(pot_filepath, "w+t", encoding="utf-8") as f:
            f.write(new_content)
        print(f"Replacing charset with UTF-8 success")
        replace_charset_ok = True
    except Exception as e:
        print(f"Replacing charset with UTF-8 failed since: {e}")

    if replace_charset_ok:
        get_po_files_ok = True
        for locale in locales_to_make:
            specific_locale_dir = os.path.normpath(os.path.join(locale_dir, locale, "LC_MESSAGES"))
            os.makedirs(specific_locale_dir, exist_ok=True)
            po_filepath = os.path.normpath(os.path.join(specific_locale_dir, f"{addon_name}.po"))
            if os.path.exists(po_filepath):
                try:
                    process = subprocess.run(
                        ["msgmerge", "--update", "--backup=off", "--no-fuzzy-matching", po_filepath, pot_filepath], 
                        capture_output=True, 
                        text=True, 
                        check=True
                    )
                    print(f"Update a po file for locale {locale} success: {po_filepath}")
                except Exception as e:
                    print(f"Update of a po file for locale {locale} failed since: {e.stderr}")
                    get_po_files_ok = False
                    break
            else:
                try:
                    process = subprocess.run(
                        ["msginit", "--no-translator", "-i", pot_filepath, "-o", po_filepath, f"--locale={locale}.UTF-8"], 
                        capture_output=True, 
                        text=True, 
                        check=True
                    )
                    print(f"Initialize a po file for locale {locale} to {po_filepath}")
                except Exception as e:
                    print(f"Initialization of a po file for locale {locale} failed since: {e.stderr}")
                    get_po_files_ok = False
                    break


In [ ]:
if get_po_files_ok:
    import polib
    translation_dict = dict()
    for locale in locales_to_make:
        specific_locale_dir = os.path.normpath(os.path.join(locale_dir, locale, "LC_MESSAGES"))
        po_filepath = os.path.normpath(os.path.join(specific_locale_dir, f"{addon_name}.po"))
        po = polib.pofile(po_filepath)
        print(f"Parse the po file for {locale}: {po_filepath}")
        locale_dict = translation_dict.setdefault(locale, {})
        for entry in po:
            if entry.msgstr and not entry.fuzzy:
                translation_dict[locale][(entry.msgctxt, entry.msgid)] = entry.msgstr
    print(translation_dict)